In [1]:
%pip install tabulate

import pandas as pd

# 1. Carregar o arquivo original
df = pd.read_excel('./dados/dados-questionario-academia.xlsx', engine='openpyxl')

# Renomear colunas para facilitar a manipulação
df.columns = [
    'timestamp', 'idade', 'estado_civil', 'periodo', 
    'frequencia', 'nota_ambiente', 'nota_aparelhos', 
    'nota_professores', 'nps'
]

# --- CRIAÇÃO DA TABELA DE PERFIL DE TREINO (DIMENSÃO) ---
# Identifica combinações únicas de Periodo e Frequencia para criar o catálogo
tb_perfil_treino = df[['periodo', 'frequencia']].drop_duplicates().reset_index(drop=True)
# Cria o ID (Primary Key)
tb_perfil_treino['id_treino'] = tb_perfil_treino.index + 1
# Reordenar colunas
tb_perfil_treino = tb_perfil_treino[['id_treino', 'periodo', 'frequencia']]

# --- PREPARAÇÃO PARA AS OUTRAS TABELAS ---
# Trazemos o id_treino para a tabela principal para poder usar nas outras
df_completo = df.merge(tb_perfil_treino, on=['periodo', 'frequencia'], how='left')

# --- CRIAÇÃO DA TABELA DE ALUNO (DIMENSÃO) ---
# Como os dados são anônimos, vamos assumir que cada resposta é um aluno distinto (surrogate key)
tb_aluno = df_completo[['idade', 'estado_civil']].copy()
tb_aluno['id_aluno'] = tb_aluno.index + 1
# Reordenar colunas
tb_aluno = tb_aluno[['id_aluno', 'idade', 'estado_civil']]

# --- CRIAÇÃO DA TABELA DE PESQUISA (FATO) ---
tb_pesquisa = df_completo.copy()
# Adiciona o ID do aluno (que criamos baseados no index na etapa anterior)
tb_pesquisa['id_aluno'] = tb_pesquisa.index + 1
# Cria o ID da pesquisa
tb_pesquisa['id_pesquisa'] = tb_pesquisa.index + 1

# Seleciona apenas as colunas necessárias (IDs e Notas)
colunas_finais_pesquisa = [
    'id_pesquisa', 'id_aluno', 'id_treino', 'timestamp',
    'nota_ambiente', 'nota_aparelhos', 'nota_professores', 'nps'
]
tb_pesquisa = tb_pesquisa[colunas_finais_pesquisa]

# --- EXIBIÇÃO E SALVAMENTO ---
print("Amostra TB_PERFIL_TREINO:")
print(tb_perfil_treino.head().to_markdown(index=False))
print("\nAmostra TB_ALUNO:")
print(tb_aluno.head().to_markdown(index=False))
print("\nAmostra TB_PESQUISA:")
print(tb_pesquisa.head().to_markdown(index=False))

# Salvar os arquivos
tb_perfil_treino.to_csv('./dados/TB_PERFIL_TREINO.csv', index=False)
tb_aluno.to_csv('./dados/TB_ALUNO.csv', index=False)
tb_pesquisa.to_csv('./dados/TB_PESQUISA.csv', index=False)

Note: you may need to restart the kernel to use updated packages.
Amostra TB_PERFIL_TREINO:
|   id_treino | periodo   | frequencia                 |
|------------:|:----------|:---------------------------|
|           1 | Manhã     | 3 a 4 vezes por semana     |
|           2 | Tarde     | 3 a 4 vezes por semana     |
|           3 | Manhã     | 1 a 2 vezes por semana     |
|           4 | Manhã     | 5 ou mais vezes por semana |
|           5 | Noite     | 3 a 4 vezes por semana     |

Amostra TB_ALUNO:
|   id_aluno |   idade | estado_civil   |
|-----------:|--------:|:---------------|
|          1 |      19 | Solteiro(a)    |
|          2 |      27 | Casado(a)      |
|          3 |      31 | Solteiro(a)    |
|          4 |      18 | Solteiro(a)    |
|          5 |      22 | Divorciado(a)  |

Amostra TB_PESQUISA:
|   id_pesquisa |   id_aluno |   id_treino | timestamp                  |   nota_ambiente |   nota_aparelhos |   nota_professores |   nps |
|--------------:|-----------:|----